In [1]:
import pandas as pd
import numpy as np
import gzip
import json
import re
import os
import sys
import prince
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [14]:
df = pd.read_csv('small.csv')

In [16]:
df

,tag1,tag2,tag3,tag4,tag5,isbn
0,historical-fiction,mystery,historical-mystery,medieval,historical,184737297X
1,short-stories,fiction,french,français,literature,0440083087
2,horror,stephen-king,fiction,sci-fi,science-fiction,0142427896
3,non-fiction,society,politics,history,economics,1400041694
4,fiction,christian-fiction,christian,giveaways,shelfari-medical,1463650132
5,mystery,england,mysteries,mood-comforting,milieu-small-town,0854563903
6,classics,literature,classic,american-literature,rory-gilmore-challenge,0743564421
7,econimics-and-finance,business,poleco,behaviour-science,risk-management,1118727010
8,fantasy,young-adult,ya,paranormal,series,0698143760
9,mystery,crime,fiction,mysteries,cornwall,0752844458


In [18]:
u_tags = pd.unique(df.iloc[:, 0:5].values.ravel('K'))

In [19]:
len(u_tags)

43

In [32]:
# add new columns from unique tags
def add_columns(df, columns):
    newcols = {}
    for i in range(len(columns)):
        newcols['%s' % columns[i]] = [0]*10
    return pd.concat([df, pd.DataFrame(newcols, index=df.index)], axis=1)

In [36]:
test = u_tags

dftags = add_columns(df, test)
dftags.drop(df.iloc[:, 0:5], axis = 1, inplace = True) 

In [37]:
dftags

,isbn,historical-fiction,short-stories,horror,non-fiction,fiction,mystery,classics,econimics-and-finance,fantasy,...,paranormal,historical,science-fiction,economics,shelfari-medical,milieu-small-town,rory-gilmore-challenge,risk-management,series,cornwall
0,184737297X,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0440083087,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0142427896,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1400041694,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1463650132,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0854563903,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0743564421,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1118727010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0698143760,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0752844458,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
df

,tag1,tag2,tag3,tag4,tag5,isbn
0,historical-fiction,mystery,historical-mystery,medieval,historical,184737297X
1,short-stories,fiction,french,français,literature,0440083087
2,horror,stephen-king,fiction,sci-fi,science-fiction,0142427896
3,non-fiction,society,politics,history,economics,1400041694
4,fiction,christian-fiction,christian,giveaways,shelfari-medical,1463650132
5,mystery,england,mysteries,mood-comforting,milieu-small-town,0854563903
6,classics,literature,classic,american-literature,rory-gilmore-challenge,0743564421
7,econimics-and-finance,business,poleco,behaviour-science,risk-management,1118727010
8,fantasy,young-adult,ya,paranormal,series,0698143760
9,mystery,crime,fiction,mysteries,cornwall,0752844458


In [ ]:
def get_tag_counts_per_book(df, unique_tags):
    